# 基于深度学习的文本分类

使用基于mindspore框架的自然语言处理库mindnlp

主仓库地址：https://github.com/mindspore-lab/mindnlp

# 构造数据集

根据所提供的news20数据集，

In [3]:
"""
news20 load function
"""
import os
import mindspore
from tqdm import tqdm
from mindspore.dataset import GeneratorDataset

mindspore.set_context(device_target="GPU") # set GPU

label_dict = {}
label_count = 0

class News20:
    """
    NEWS dataset source
    """

    def __init__(self, path)->None:
        self.path: str = path
        self._text, self._label = [], []
        self.label_dict = {}
        self.label_count = 0
        self._load()

    def _load(self):
        labels = os.listdir(self.path)
        for label in labels:
            self.label_dict[label] = self.label_count
            self.label_count += 1
            label_path = os.path.join(self.path, label)
            with tqdm(total = len(os.listdir(label_path)), desc = label) as pbar:
                for t in os.listdir(label_path):
                    pbar.update(1)
                    text_path = os.path.join(label_path, t)
                    file = open(text_path, "r" ,encoding="latin-1") # latin-1
                    text = file.read().strip()
                    begin = text.find('\n\n')
                    if 0 < begin: # skip head
                        text = text[begin:]
                    self._text.append(text)
                    self._label.append(self.label_dict[label])

    def __getitem__(self, index):
        return self._text[index], self._label[index]

    def __len__(self):
        return len(self._label)

def news20(source:News20):
    r"""
    Load the news20 dataset
    """
    column_names = ["text", "label"]
    return GeneratorDataset(source = source, column_names = column_names, shuffle = False) # 不shuffle，划分数据集时会进行打乱


In [4]:
# 载入数据
dataset_source = News20(r"/run/determined/workdir/qishunheng/lijiaming/nlp_ex/dataset/20_newsgroup")
print(dataset_source.label_dict)
dataset = news20(dataset_source)

comp.sys.ibm.pc.hardware: 100%|██████████| 1000/1000 [00:01<00:00, 541.61it/s]

{'sci.electronics': 0, 'misc.forsale': 1, 'rec.sport.hockey': 2, 'rec.autos': 3, 'alt.atheism': 4, 'comp.sys.mac.hardware': 5, 'talk.politics.mideast': 6, 'sci.med': 7, 'rec.motorcycles': 8, 'soc.religion.christian': 9, 'talk.politics.guns': 10, 'sci.crypt': 11, 'rec.sport.baseball': 12, 'comp.windows.x': 13, 'talk.religion.misc': 14, 'sci.space': 15, 'comp.os.ms-windows.misc': 16, 'comp.graphics': 17, 'talk.politics.misc': 18, 'comp.sys.ibm.pc.hardware': 19}


In [5]:
# define Models & Loss & Optimizer
hidden_size = 256
output_size = 20
num_layers = 2
bidirectional = False
drop = 0.5
lr = 0.001

In [6]:
# 数据预处理
import mindspore
from mindspore.dataset import text, transforms
from mindnlp.modules import Glove
from mindnlp.dataset.transforms import BasicTokenizer
from mindnlp.dataset import process

def data_process(dataset, tokenizer, vocab, batch_size = 64, max_len = 1000, drop_remainder = False):
    """
    预处理
    """
    # print("!!!!!!!!!!!!!!!!!!!!!!!")
    dataset = dataset.map([tokenizer], 'text')
    lookup_op = text.Lookup(vocab, unknown_token = '<unk>')
    dataset = dataset.map([lookup_op], 'text')
    pad_value = vocab.tokens_to_ids('<pad>')
    pad_op = transforms.PadEnd([max_len], pad_value)
    dataset = dataset.map([pad_op], 'text')
    onehot_op = transforms.OneHot(num_classes=20)
    dataset = dataset.map([onehot_op], 'label')
    type_cast_op = transforms.TypeCast(mindspore.float32)
    dataset = dataset.map([type_cast_op], 'label')

    dataset = dataset.batch(batch_size, drop_remainder = drop_remainder)
    return dataset

tokenizer = BasicTokenizer(True)
embedding, vocab = Glove.from_pretrained('6B', 100, special_tokens=["<unk>", "<pad>"], dropout=drop, root = r"/run/determined/workdir/qishunheng/lijiaming")

dataset_process = data_process(dataset, tokenizer, vocab) # 其余使用默认参数

In [7]:
# 划分数据集
train_dataset, test_dataset = dataset_process.split([0.8, 0.2])

In [8]:
# 构建网络
import math

from mindspore import nn
from mindspore import ops
from mindspore.common.initializer import Uniform, HeUniform
from mindnlp.abc import Seq2vecModel

class Head(nn.Cell):
    """
    Head for Classification model
    """
    def __init__(self, hidden_dim, output_dim, dropout):
        super().__init__()
        weight_init = HeUniform(math.sqrt(5))
        bias_init = Uniform(10 / math.sqrt(hidden_dim * 2))
        self.fc = nn.Dense(hidden_dim * 2, output_dim, weight_init=weight_init, bias_init=bias_init)
        self.softmax = nn.Softmax()
        self.dropout = nn.Dropout(1 - dropout)

    def construct(self, context):
        context = ops.concat((context[-2, :, :], context[-1, :, :]), axis=1)
        context = self.dropout(context)
        return self.softmax(self.fc(context))


class Classification(Seq2vecModel):
    """
    Classification model
    """
    def __init__(self, encoder, head):
        super().__init__(encoder, head)
        self.encoder = encoder
        self.head = head

    def construct(self, text):
        _, (hidden, _), _ = self.encoder(text)
        output = self.head(hidden)
        return output

In [ ]:
from mindnlp.modules import RNNEncoder
from mindnlp.engine.metrics import Accuracy
from mindnlp.engine.trainer import Trainer

lstm_layer = nn.LSTM(100, hidden_size, num_layers=num_layers, batch_first=True,
                     dropout=drop, bidirectional=bidirectional)
encoder = RNNEncoder(embedding, lstm_layer)
head = Head(hidden_size, output_size, drop)

net = Classification(encoder, head)
loss = nn.CrossEntropyLoss()
optimizer = nn.Adam(net.trainable_params(), learning_rate=lr)

# define metrics
metric = Accuracy()

# define trainer
trainer = Trainer(network=net, train_dataset = train_dataset, eval_dataset = test_dataset, metrics=metric,
                  epochs=5, loss_fn=loss, optimizer=optimizer)
trainer.run(tgt_columns="label", jit=True)
print("end train")

Epoch 0:   0%|          | 0/250 [00:00<?, ?it/s][WARNING] ME(77922:139915739633472,MainProcess):2022-12-16-17:41:24.259.811 [mindspore/ops/primitive.py:710] The "_check_is_tensor" is a constexpr function. The input arguments must be all constant value.
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda-11.1/lib64/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
Epoch 0:  45%|████▍     | 112/250 [00:19<00:15,  8.83it/s, loss=2.9969065]

: 

: 